In [24]:
import joblib
import re
import string
import jieba
import pandas as pd
from tokenizers import (
    models,
    normalizers,
    pre_tokenizers,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm import tqdm
tqdm.pandas()
from transformers import PreTrainedTokenizerFast
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [25]:
# 请修改测试集文件名
test_dir = 'test.csv'

In [26]:
# 如测试集中不存在topic列，请使用 test = df[['content']]
df = pd.read_csv(test_dir)
test = df[['content', 'topic']]
#test = df[['content']]

In [27]:
def get_stopword():
    with open('baidu_stopwords.txt', 'r', encoding='utf-8') as f:
        stopwords = set(line.strip() for line in f)
    stopwords.update(["年","月","日"])
    return stopwords

def wordopt_cn(text):
    # replace punctuation with space
    no_punct = re.sub(pattern, ' ', text)
    return str(no_punct)

def remove_stopwords(words):
    return (word for word in words if word not in stopwords)

def process_content(content):
    content = wordopt_cn(content)
    words = jieba.cut(content)
    words = remove_stopwords(words)
    content = ' '.join(words)
    content = re.sub(spaces_pattern, ' ', content)
    return content

# Precompile the regular expressions
punctuation = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏." + string.punctuation
pattern = re.compile('[%s]' % re.escape(punctuation))
spaces_pattern = re.compile(r'\s+')
stopwords = get_stopword()

In [28]:
test['original_content'] = test['content']
test['content'] = test['content'].progress_apply(process_content)

C:\Users\shens\AppData\Local\Temp\ipykernel_22252\2199731547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['original_content'] = test['content']
100%|██████████| 3405/3405 [00:11<00:00, 305.48it/s]
C:\Users\shens\AppData\Local\Temp\ipykernel_22252\2199731547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['content'] = test['content'].progress_apply(process_content)


In [29]:
def dummy(text):
    return text
#test=pd.read_csv('test_jieba.csv')
vectorizer = joblib.load('vectorizer.pkl')
model = joblib.load('ensemble.pkl')

In [30]:
LOWERCASE = False
VOCAB_SIZE = 100_000
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)

dataset = Dataset.from_pandas(test[['content']])


def train_corp_iter():
    for i in range(0, len(dataset), 1000):
        yield dataset[i: i + 1000]["content"]


raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(test['content'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))
    
tf_test = vectorizer.transform(tokenized_texts_test)

100%|██████████| 3405/3405 [00:02<00:00, 1661.39it/s]


In [31]:
pred = model.predict(tf_test)
prob = model.predict_proba(tf_test)
test["pred"] = pred
test["prob"] = prob.max(axis=1)

C:\Users\shens\AppData\Local\Temp\ipykernel_22252\3318733313.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pred"] = pred


In [32]:
accuracy = accuracy_score(test['topic'], test['pred'])
precision = precision_score(test['topic'], test['pred'], average='weighted')
recall = recall_score(test['topic'], test['pred'], average='weighted')
f1 = f1_score(test['topic'], test['pred'], average='weighted')
print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1: {f1}')

Accuracy: 0.9750367107195301
Precision: 0.9752436834777642
Recall: 0.9750367107195301
F1: 0.9750324678322444


In [33]:
low_prob = test[test['prob'] < 0.5]
low_prob

,content,topic,original_content,pred,prob
40,2023 亚太经合 会 APEC 领袖 峰会 今 15 日于 美国 旧金山 登场 泰国 总理...,1,2023年亚太经合会APEC领袖峰会今(15)日于美国旧金山登场，泰国总理赛塔(Settha...,0,0.498137
82,美国 已 中 俄 发出 邀请 普京 本人 去不去 俄罗斯 会 悬念 留 亚太经合组织 APE...,1,美国已向中俄发出邀请，普京本人去不去，俄罗斯会把悬念留到最后今年的亚太经合组织（APEC）峰...,3,0.366565
148,微软 采购 核聚变 初创 公司 Helion 电力 Helion 预计 2028 年前 上线...,2,微软将采购由核聚变初创公司Helion产生的电力，Helion预计2028年前上线50兆瓦核...,0,0.380416
252,2023 新德里 G20 领导人 峰会 成员国 名单 特邀 成员国 名单 印度 拟定 20 ...,3,2023年在新德里举行的G20领导人峰会成员国名单以及特邀成员国名单，印度已经拟定出来，除了...,3,0.460188
261,印度 新德里 G20 峰会 落幕 国际 政坛 落幕 各国 领导人 展现 风采 机会 英国 头...,3,印度新德里G20峰会的落幕，不仅是国际政坛的落幕，也是各国领导人展现风采的机会。英国头号夫人...,3,0.444316
...,...,...,...,...,...
3061,2023 神舟 16 号 发射成功 观后感 三 二 点火 火团 喷涌 而出 小小的 屏幕 画...,21,2023年神舟16号发射成功观后感“三、二、一，点火”。火团喷涌而出。望着小小的屏幕，我看到...,19,0.304691
3075,神舟 十六号 发射 全过程 47002023 05 30 11 07 43 未经 作者 授权...,21,神舟十六号_发射全过程47002023-05-30 11:07:43未经作者授权，禁止转载9...,21,0.274267
3099,2023 12 31 日晚 国家 主席 习近平 发表 二 〇 二 四年 新年贺词 2013 ...,21,2023年12月31日晚，国家主席习近平发表二〇二四年新年贺词。自2013年岁末起，习近平主...,19,0.490948
3118,点击 右上角 微信 好友 朋友圈 请 浏览器 分享 功能 分享 导航 时政 国际 时评 理论...,21,点击右上角微信好友朋友圈请使用浏览器分享功能进行分享全部导航时政国际时评理论文化科技教育经济...,21,0.443351


In [34]:
from sklearn.metrics.pairwise import cosine_similarity
from text2vec import SentenceModel
from tqdm import tqdm
tqdm.pandas()
import numpy as np
m = SentenceModel("shibing624/text2vec-base-chinese-paraphrase", max_seq_length=512, device="cuda")
cls_emb_center = joblib.load("cls_emb_center_512.pkl")

test_emb = [m.encode(i) for i in tqdm(low_prob['original_content'])]

preds = []
max_sim = []
for i in test_emb:
    sim = [cosine_similarity(i.reshape(1, -1), cls_emb_center[j].reshape(1, -1))[0][0] for j in range(24)]
    max_sim.append(max(sim))
    pred = np.argmax(sim)
    preds.append(pred)

low_prob['pred'] = preds
low_prob["max_sim"] = max_sim

2024-03-21 18:10:16.445 | DEBUG    | text2vec.sentence_model:__init__:80 - Use device: cuda
100%|██████████| 64/64 [00:02<00:00, 25.59it/s]
C:\Users\shens\AppData\Local\Temp\ipykernel_22252\3972939978.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_prob['pred'] = preds
C:\Users\shens\AppData\Local\Temp\ipykernel_22252\3972939978.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_prob["max_sim"] = max_sim


In [35]:
low_prob_diff = low_prob[low_prob['topic'] != low_prob['pred']]
low_prob_diff

,content,topic,original_content,pred,prob,max_sim
82,美国 已 中 俄 发出 邀请 普京 本人 去不去 俄罗斯 会 悬念 留 亚太经合组织 APE...,1,美国已向中俄发出邀请，普京本人去不去，俄罗斯会把悬念留到最后今年的亚太经合组织（APEC）峰...,4,0.366565,0.879174
148,微软 采购 核聚变 初创 公司 Helion 电力 Helion 预计 2028 年前 上线...,2,微软将采购由核聚变初创公司Helion产生的电力，Helion预计2028年前上线50兆瓦核...,0,0.380416,0.817164
1256,苏联 历次 对外 战争 理由 开战 方式 梳理 无 处天 战争 一个 国家 来说 生死攸关 ...,11,苏联历次对外战争的理由及开战方式梳理无所有处天战争对于一个国家来说，往往是生死攸关的大事，所...,6,0.300657,0.883002
1285,叙利亚 驻华大使 一带 一路 使 重建 中 叙利亚 受益匪浅 2022 1 12 中国 叙利...,11,叙利亚驻华大使：加入“一带一路”将使重建中的叙利亚受益匪浅。2022年1月12日，中国和叙利...,22,0.489235,0.888616
1450,台湾 国防部 12 24 日称 24 小时 中 歼 10 歼 11 歼 16 各型 共 8 ...,12,台湾国防部12月24日称，在过去24小时中，又有歼-10、歼-11、歼-16等各型共8架战机...,19,0.281869,0.808087
1888,新 时代 中国 山水 披绿纱 引来 德企 大当家 来自 德国 柯雪婷 Kerstin Kae...,0,新时代，我在中国 | 山水披绿纱，引来德企大当家来自德国的柯雪婷（Kerstin Kaehl...,22,0.385803,0.786594
1929,10 月份 我国 自然灾害 台风 洪涝灾害 风雹 低温 冷冻 雪灾 地震 地质灾害 森林 火...,0,10月份，我国自然灾害以台风、洪涝灾害为主，风雹、低温冷冻、雪灾、地震、地质灾害和森林火灾等...,18,0.414659,0.899937
1967,5 6 省厅 印发 《 做好 2023 年度 防汛防台 风 工作 通知 全力 做好 防汛防台...,0,5月6日，省厅印发《关于做好2023年度防汛防台风工作的通知》，就全力做好防汛防台风，确保人...,17,0.368595,0.875500
2072,网络安全 自然 不 例外 几年 人工智能 热度 很 高 一个 话题 AlphaGo 横空出世...,0,网络安全自然也不例外。最近几年，人工智能是热度很高的一个话题。AlphaGo的横空出世，以及...,2,0.465266,0.868130
2094,近日 中央 网信 办 主办 2023 中国 正 能量 网络 精品 征集 展播 活动 正式 启...,0,近日，由中央网信办主办的2023中国正能量网络精品征集展播活动正式启动。活动以“凝聚强国力量...,7,0.445021,0.801917


In [36]:
test.update(low_prob[['pred']])
accuracy = accuracy_score(test['topic'], test['pred'])
precision = precision_score(test['topic'], test['pred'], average='weighted')
recall = recall_score(test['topic'], test['pred'], average='weighted')
f1 = f1_score(test['topic'], test['pred'], average='weighted')
print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1: {f1}')

Accuracy: 0.9765051395007343
Precision: 0.9766608344068973
Recall: 0.9765051395007343
F1: 0.9765037577749967


In [37]:
diff = test[test['topic'] != test['pred']]
diff

,content,topic,original_content,pred,prob
0,2023 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人...,1,2023年，习近平主席亲自擘画引领，中国特色大国外交扎实推进。在推动构建人类命运共同体的崇高...,22,0.698144
7,APEC 峰会 前 公使 提出 四点 核心 利益 中 美 会晤 看 美 表现 一带 一路 国...,1,APEC峰会前，我公使提出四点核心利益，中美是否会晤，得看美表现“一带一路”国际高峰论坛取得...,4,0.511553
43,焦点访谈 把舵 中美关系 领航 亚太 合作 央视网 2023 11 19 22 01 加载 ...,1,焦点访谈：把舵中美关系 领航亚太合作央视网2023年11月19日 22:01正在加载央视网消...,4,0.792356
60,亚太 国家 地区 领导人 13 抵达 美国 亚太经合组织 APEC 第三十次 领导人 非正式...,1,随着亚太一些国家和地区领导人从13日开始抵达美国，亚太经合组织（APEC）第三十次领导人非正...,4,0.731006
82,美国 已 中 俄 发出 邀请 普京 本人 去不去 俄罗斯 会 悬念 留 亚太经合组织 APE...,1,美国已向中俄发出邀请，普京本人去不去，俄罗斯会把悬念留到最后今年的亚太经合组织（APEC）峰...,4,0.366565
...,...,...,...,...,...
3096,2022 11 28 上午 9 时 神舟 十五号 载人 飞行 新闻 发布会 酒泉卫星发射中心...,21,2022年11月28日上午9时，神舟十五号载人飞行任务新闻发布会在酒泉卫星发射中心召开，中国...,19,0.925044
3099,2023 12 31 日晚 国家 主席 习近平 发表 二 〇 二 四年 新年贺词 2013 ...,21,2023年12月31日晚，国家主席习近平发表二〇二四年新年贺词。自2013年岁末起，习近平主...,19,0.490948
3127,神舟 十七号 发射成功 续写 飞天 梦想 建设 航天 强国 北京 时间 10 26 11 时...,21,神舟十七号发射成功：续写飞天梦想，建设航天强国。北京时间10月26日11时13分59秒，搭载...,20,0.946573
3132,神舟 十六号 出舱 多个 首次 神 十七 10 发射 航天员 杨利伟 7 国外 航天 执行 ...,21,神舟十六号出舱，有多个首次！神十七10月发射，有航天员杨利伟？7月，国外的航天任务执行可能最...,20,0.714251
